In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
#Use "-1" for CPU-only devices

#If working on GPU enviroment
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
#Limit the memory allocated by tensorflow to only the necessary amount

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['PYTHONHASHSEED'] = '2'

In [2]:
import numpy as np
from numpy.random import seed
seed(18)

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.random import set_seed
from tensorflow.keras.utils import set_random_seed
set_seed(18)
set_random_seed(18)

import keras

import matplotlib.pyplot as plt

from Supporting_func_file import *

Version_check()

/usr/lib/python3.11/site-packages/h5py/__init__.py:36: UserWarning: h5py is running against HDF5 1.14.1 when it was built against 1.14.0, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


All packages are using the targeted versions.


## Preparing the data
#### Loading the 30,000 measured samples

In [3]:
file_path = "Data_sets/Performance_evaluation/Performance_evaluation_50_ms_data.npz"

fiber_bundle_counts, input_states_rho = Load_data_file(file_path)

print("Shapes")
print("Count distribution of 7 detectors:           ", fiber_bundle_counts.shape)
print("Corresponding coherence matrices:            ", input_states_rho.shape)

Shapes
Count distribution of 7 detectors:            (30000, 7)
Corresponding coherence matrices:             (30000, 2, 2)


#### Generating the mixed state samples

In [4]:
number_of_repetitions = 20
#The created dataset will consists of (number_of_repetitions * 30000) generated mixed data covering the whole Bloch sphere

radial_positions = Radius_dist((number_of_repetitions, input_states_rho.shape[0]))
#Supporting variable for generating mixed states

data_rhos, data_counts = Generate_mixed_dataset(input_states_rho, fiber_bundle_counts, radial_positions)
#Dataset of coherence matrices (data_rhos) with corresponding count distributions (data_counts) generated from the 30,000 measured samples

#### Pre-processing the data for the neural network

In [5]:
data_in = Probability_norm(data_counts)
#Network inputs are the count distributions of the polarization state normed to represent the relative frequencies of the 7 detection channels. I.e., for first polarization state, sum(data_in[0,:7]) = 1.

data_out = Tau_to_Flat(np.linalg.cholesky(data_rhos))
#Network outputs are the "Tau" Cholesky decomposition (in a NN-friendly version "Flat") of the coherence matrix Rho.

#Example to explain the data form
print("First coherency matrix:")
print(data_rhos[0].round(3))
print("")
print("Its Cholesky decomposition in the form of a triangular matrix, Tau:")
print(np.linalg.cholesky(data_rhos[0]).round(3))
print("")
print("The 4 real-valued elements of the Cholesky decomposition, Flat:")
print(data_out[0].round(3))

First coherency matrix:
[[0.497+0.j   0.303-0.31j]
 [0.303+0.31j 0.503-0.j  ]]

Its Cholesky decomposition in the form of a triangular matrix, Tau:
[[0.705+0.j    0.   +0.j   ]
 [0.43 +0.439j 0.354+0.j   ]]

The 4 real-valued elements of the Cholesky decomposition, Flat:
[0.705 0.43  0.439 0.354]


In [6]:
#Randomly shuffling the dataset. Keeping the position relevance. I.e., data_in[i] corresponds to data_out[i] after shuffle.
seed(18)
np.random.shuffle(data_in)
seed(18)
np.random.shuffle(data_out)

In [7]:
#Separating the dataset into training, validation, and test set
first_borderline = (number_of_repetitions+1) * 16000
second_borderline = (number_of_repetitions+1) * 24000

train_in = data_in[:first_borderline]
train_out = data_out[:first_borderline]
#Training set was used to train the network

val_in = data_in[first_borderline:second_borderline]
val_out = data_out[first_borderline:second_borderline]
#Validation set was used to optimize the network

test_in = data_in[second_borderline:]
test_out = data_out[second_borderline:]
#Test set is used now to evaluate the performance

print("Number of training data:  ", train_in.shape)
print("Number of validation data:", val_in.shape)
print("Number of test data:      ", test_in.shape)

Number of training data:   (336000, 7)
Number of validation data: (168000, 7)
Number of test data:       (96000, 7)


## Evaluating the model
#### Loading the trained model

In [8]:
model = load_model("Trained_models/Main_results_model.h5", 
                   custom_objects={"fidelity_metric": fidelity_metric})

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 250)               2000      
                                                                 
 dropout (Dropout)           (None, 250)               0         
                                                                 
 batch_normalization (BatchN  (None, 250)              1000      
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 250)               62750     
                                                                 
 dropout_1 (Dropout)         (None, 250)               0         
                                                                 
 batch_normalization_1 (Batc  (None, 250)              1000      
 hNormalization)                                        

#### Evaluating the predictions

In [9]:
pred_test_rho = Flat_to_Density(model.predict(test_in))       #Predicted polarization states in the form of coherence matrices
true_test_rho = Flat_to_Density(test_out)                     #The targeted coherence matrices

test_fidelity_array = Fidelity(true_test_rho, pred_test_rho)
test_infidelity_array = (1-test_fidelity_array)
#The fidelities and infidelities between predicted and targeted coherence matrices

3000/3000 [==============================] - 6s 2ms/step


In [10]:
print("Infidelity values used in the main text:")
print("The average value:  ", np.mean(test_infidelity_array).round(6))
print("The 10th percentile:", np.quantile(test_infidelity_array, 0.1).round(6))
print("The 95th percentile:", np.quantile(test_infidelity_array, 0.9).round(6))

Infidelity values used in the main text:
The average value:   0.000838
The 10th percentile: 5.3e-05
The 95th percentile: 0.001829
